In [1]:
import pandas as pd
from PyPDF2 import PdfReader
import re

In [2]:
df = pd.DataFrame(columns=['location', 'ref', 'price', 'description', 'owner', 'year'])

In [4]:
def extract_all_data(pdf, year):
    for page_num in range(len(pdf.pages)):
        pageObj = pdf.pages[page_num]  # Turn page into object
        text = pageObj.extract_text()
        if (len(text)>0):
            split_text = text.split("\n")
            
            location = ''
            ref = ''
            price = ''
            description = ''
            owner = ''
            for line in split_text:

                if (owner != ''):
                    df.loc[len(df.index)] = [location, ref, price, description, owner, year] 
                    location = ''
                    ref = ''
                    price = ''
                    description = ''
                    owner = ''     

                if 'www.ownersbest.com.mt' in line:
                    continue

                if 'CALL on 2149 2299 or 7949 2299 to SELL FAST' in line:
                    continue

                if (location == ''):
                    location = line
                    continue
                
                if 'REF' in line:
                    split_ref = line.split('€')
                    ref = split_ref[0]
                    if (len(split_ref)>1):
                        price = split_ref[1]
                    continue

                if re.search(r"Owner \d+/\d+", line):
                    owner = line
                    continue

                if (location!='' and ref !=''):
                    description = description + (line + ' ')
                    continue


In [5]:
files = [{"file": 'ob-april-2016.pdf', "year": 2016}, 
         {"file":'ob-jan2014-s.pdf', "year":2014}, 
         {"file":'ob-june2012.pdf', "year":2012},
         {"file":'ownersbest-oct2011.pdf', "year":2011},
         {"file":'ob-2015.pdf', "year":2015},
         {"file":'sep13-ob.pdf', "year":2013}]


In [6]:
for file in files:
    print ('Extracting {}'.format(file))
    extract_all_data(PdfReader(file['file']), file['year'])

Extracting {'file': 'ob-april-2016.pdf', 'year': 2016}
Extracting {'file': 'ob-jan2014-s.pdf', 'year': 2014}
Extracting {'file': 'ob-june2012.pdf', 'year': 2012}
Extracting {'file': 'ownersbest-oct2011.pdf', 'year': 2011}
Extracting {'file': 'ob-2015.pdf', 'year': 2015}
Extracting {'file': 'sep13-ob.pdf', 'year': 2013}


In [7]:
df.to_csv('data_2011_2016.csv', mode='w', index=None)

In [8]:
df = pd.read_csv("./data_2011_2016.csv")
df.head()

,location,ref,price,description,owner,year
0,Xghajra,REF 571082,"256,000/Lm109,901",TERRACED HOUSE minutes walk from the promenade...,Owner 79728192/21806554,2016
1,Swieqi,REF 571088,"365,000/Lm156,695","TERRACED HOUSE in great area, having hall lead...",Owner 79380053/21385502,2016
2,Qormi,REF 571239,"300,000/Lm128,790","Highly finished elevated TERRACED HOUSE, consi...",Owner 77187944/21437467,2016
3,Msida,REF 570664,"186,300/Lm79,979","Corner, well kept TERRACED HOUSE comprising an...",Owner 99450405/21246533,2016
4,San Gwann,REF 571734,"210,000/Lm90,153",Corner TERRACED HOUSE comprising entrance hall...,Owner 99427561/99883181,2016


In [9]:
df.drop('owner', axis=1, inplace=True)

In [10]:
df['type'] = df['description'].apply(lambda description: ' '.join(re.findall(r'[A-Z]{2,}', str(description))))
df['type'] = df['type'].apply(lambda text: ' '.join(set(text.split())))


In [447]:
df.loc[df['type'].str.contains('PLOT', case=False, na=False), 'type'] = 'PLOT'
df.loc[df['type'].str.contains('APARTMENT', case=False, na=False), 'type'] = 'APARTMENT'
df.loc[df['type'].str.contains('HOUSE', case=False, na=False), 'type'] = 'HOUSE'
df.loc[df['type'].str.contains('VILLA', case=False, na=False), 'type'] = 'VILLA'
df.loc[df['type'].str.contains('MAISONETTE', case=False, na=False), 'type'] = 'MAISONETTE'
df.loc[df['type'].str.contains('PENTHOUSE', case=False, na=False), 'type'] = 'PENTHOUSE'
df.loc[df['type'].str.contains('GARAGE', case=False, na=False), 'type'] = 'GARAGE'
df.loc[df['type'].str.contains('COMMERCIAL', case=False, na=False), 'type'] = 'COMMERCIAL'
df.loc[df['type'].str.contains('FARM', case=False, na=False), 'type'] = 'FARM'
df.loc[df['type'].str.contains('BUSINESS PREMISES', case=False, na=False), 'type'] = 'COMMERCIAL'
df.loc[df['type'].str.contains('BUNGALOW', case=False, na=False), 'type'] = 'BUNGALOW'
df.loc[df['type'].str.contains('GROCERY', case=False, na=False), 'type'] = 'COMMERCIAL'


In [11]:
df['type'].unique()

array(['TERRACED HOUSE', 'VILLA BBQ', 'HOUSE CHARACTER OF',
       'TOWNHOUSE HOUSE CHARACTER OF', 'TOWNHOUSE', 'TOWNHOUSE HA',
       'MAISONETTE', 'MAISONETTE EUR', 'MAISONETTE AC',
       'DUPLEX MAISONETTE SOLITARY', 'DUPLEX MAISONETTE',
       'MAISONETTE SOLITARY', 'MAISONETTE SOLITARY BBQ', 'PENTHOUSE',
       'DUPLEX PENTHOUSE', 'APARTMENT', 'APARTMENT AC',
       'DUPLEX APARTMENT', 'APARTMENT BBQ', 'APARTMENT STUDIO',
       'SITE GARAGE', 'GARAGE', 'APARTMENTS PLOT',
       'DETACHED HOUSE VILLA SEMI TERRACED', 'SEMI MAISONETTE DETACHED',
       '', 'SHOP COMMERCIAL', 'COMMERCIAL STORAGE', 'COMMERCIAL', 'FARM',
       'TOWNHOUSE AC HOUSE TOWN', 'HOUSE CHARACTER FARMHOUSE OF',
       'APARTMENTS', 'APARTMENT APARTMENTS', 'APARTMENT EUR',
       'DUPLEX MAISONETTE SOLITARY AC', 'DUPLEX MAISONETTE AC',
       'SITE STORE GARAGE', 'STORE GARAGE', 'AIRSPACE MAISONETTE SITE',
       'PLOT', 'BUSINESS PREMISES', 'OFFICE', 'TERRACED HOUSE BBQ',
       'VILLA', 'BUNGALOW BBQ', 'FARMH

In [12]:
df['price'] = df['price'].str.split('/', expand=True)[0].str.replace(',', '')


In [13]:
df = df[~df['type'].isin(['GARAGE', 'PLOT', 'COMMERCIAL', 'FARM', 'OFFICE', 'LAND'])]

In [14]:
df['garage'] = df['description'].str.contains('garage')
df['balcony'] = df['description'].str.contains('balcony')
df['roof'] = df['description'].str.contains('roof')
df['garden'] = df['description'].str.contains('garden')

In [15]:
def extract_bedrooms(description):
    match = re.search(r'(\d+)(\s?.{0,6}?\s?)bedroom', description)
    return int(match.group(1)) if match else None

In [16]:
df['bedrooms'] = df['description'].apply(extract_bedrooms)
df = df[~pd.isna(df['bedrooms'])].copy()

In [17]:
df['location'] = df['location'].str.lstrip()
df['location'] = df['location'].str.rstrip()

In [18]:
df['location'] = df['location'].replace('Ghadira', 'Mellieha')
df['location'] = df['location'].replace('M''Scala', 'Marsascala')
df['location'] = df['location'].replace('Qajjenza', 'Birzebbugia')
df['location'] = df['location'].replace('San Pawl tat-Targa', 'Naxxar')
df['location'] = df['location'].replace('High Ridge', 'Swieqi')
df['location'] = df['location'].replace('Ta Giorni.', 'St Julians')

In [19]:
df['price'] = df['price'].astype(int)

In [20]:
df = df.query('type !=""').copy()

In [21]:
avg_price_year_type = df.groupby(['year', 'type'])['price'].mean().reset_index().rename(columns={'price': 'avg_price_year_type'})
df = pd.merge(df, avg_price_year_type, on=['year', 'type'], how='left')

In [413]:
# avg_price_per_year_type = df.groupby(['year', 'type'])['price'].mean().reset_index()
# avg_price_per_year_type['year'] = avg_price_per_year_type['year'] + 1
# avg_price_per_year_type = avg_price_per_year_type.rename(columns={'price': 'avg_price_last_year_type'})
# df = pd.merge(df, avg_price_per_year_type, on=['year', 'type'], how='left')


In [22]:
df['year_squared'] = df['year'] ** 2


In [23]:
df.drop_duplicates(inplace=True)

In [24]:
df.shape

(1411, 13)

In [25]:
df.head()

,location,ref,price,description,year,type,garage,balcony,roof,garden,bedrooms,avg_price_year_type,year_squared
0,Xghajra,REF 571082,256000,TERRACED HOUSE minutes walk from the promenade...,2016,TERRACED HOUSE,True,True,True,False,3.0,255987.5,4064256
1,Swieqi,REF 571088,365000,"TERRACED HOUSE in great area, having hall lead...",2016,TERRACED HOUSE,True,False,False,False,3.0,255987.5,4064256
2,Qormi,REF 571239,300000,"Highly finished elevated TERRACED HOUSE, consi...",2016,TERRACED HOUSE,True,False,True,True,2.0,255987.5,4064256
3,Msida,REF 570664,186300,"Corner, well kept TERRACED HOUSE comprising an...",2016,TERRACED HOUSE,False,True,True,False,3.0,255987.5,4064256
4,San Gwann,REF 571734,210000,Corner TERRACED HOUSE comprising entrance hall...,2016,TERRACED HOUSE,False,True,True,False,3.0,255987.5,4064256


In [26]:
# X = df.drop(['price', 'ref', 'description'], axis=1)
# y = df['price']
X = df.drop(['price', 'ref', 'description'], axis=1)
y = df['price']

In [27]:
X.head()

,location,year,type,garage,balcony,roof,garden,bedrooms,avg_price_year_type,year_squared
0,Xghajra,2016,TERRACED HOUSE,True,True,True,False,3.0,255987.5,4064256
1,Swieqi,2016,TERRACED HOUSE,True,False,False,False,3.0,255987.5,4064256
2,Qormi,2016,TERRACED HOUSE,True,False,True,True,2.0,255987.5,4064256
3,Msida,2016,TERRACED HOUSE,False,True,True,False,3.0,255987.5,4064256
4,San Gwann,2016,TERRACED HOUSE,False,True,True,False,3.0,255987.5,4064256


In [28]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split

X_copy = X.copy()
y_copy = y.copy()

model = MLPRegressor(random_state=42)

categorical_features = [ 'location', 'type']
for cat in categorical_features:
    dummies = pd.get_dummies(X_copy[cat], prefix=cat)
    X_copy = pd.concat([X_copy, dummies], axis=1)
    X_copy.drop(cat, axis=1, inplace=True)


int_features = ['garage', 'balcony', 'roof','garden']
for int_val in int_features:
    X_copy[int_val]=X_copy[int_val].astype(int)


numeric_features = ['year', 'bedrooms', 'avg_price_year_type', 'year_squared']
mm_scaler = MinMaxScaler()
for num in numeric_features:
    X_copy[num] = mm_scaler.fit_transform(X_copy[[num]])

X_copy.head()
    

X_train, X_test, y_train, y_test = train_test_split(X_copy, y_copy, test_size=0.2, random_state=42)

In [29]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

model = GradientBoostingRegressor(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.0001, 0.001],
    'max_depth': [3, 5, 7, 9, 11]
}

from sklearn.model_selection import GridSearchCV
grid_search_gb = GridSearchCV(model, param_grid, cv=5)
grid_search_gb.fit(X_train, y_train)

# Print the best parameters
print(grid_search_gb.best_params_)

# Evaluate the model
y_pred = grid_search_gb.predict(X_test)
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

{'learning_rate': 0.001, 'max_depth': 11, 'n_estimators': 300}
Root Mean Squared Error: 61581.200882466306


In [30]:
param_dist = {
    'hidden_layer_sizes': [(60,40, 1), (60,120,60,1), (60, 30, 1)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.5],
    'learning_rate': ['constant','adaptive'],
}

mlp = MLPRegressor()

rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

grid_search_mlp = GridSearchCV(mlp, param_dist, cv=5, scoring=rmse_scorer, n_jobs=-1)

grid_search_mlp.fit(X_train, y_train)

print("Best parameters found: ", grid_search_mlp.best_params_)
print("Lowest RMSE found: ", np.abs(grid_search_mlp.best_score_))

/Users/kylegrange/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kylegrange/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kylegrange/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kylegrange/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Use

Best parameters found:  {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (60, 120, 60, 1), 'learning_rate': 'adaptive', 'solver': 'adam'}
Lowest RMSE found:  59483.692288095925


/Users/kylegrange/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [31]:
from sklearn.ensemble import RandomForestRegressor

param_dist = {
    'n_estimators': [100,1000,5000],
}

rf = RandomForestRegressor()

grid_search_rf = GridSearchCV(rf, param_dist, cv=5)

grid_search_rf.fit(X_train, y_train)

print("Best parameters found: ", grid_search_rf.best_params_)
print("Lowest RMSE found: ", np.abs(grid_search_rf.best_score_))

Best parameters found:  {'n_estimators': 5000}
Lowest RMSE found:  0.5857657211984004


In [32]:
prediction = grid_search_rf.predict(X_test)
mse = mean_squared_error(y_test, prediction)
rmse = mse**.5
print(rmse)

44820.06068022985
